In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import time

model = load_model("drowsiness_lenet_model.h5")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

cap = cv2.VideoCapture(0)

frozen_frame = None  
closed_eye_counter = 0  
threshold = 40

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    if len(faces) == 0:
        if frozen_frame is None:
            frozen_frame = frame.copy()
        cv2.putText(frozen_frame, "No face detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Drowsiness Detection", frozen_frame)
    else:
        frozen_frame = None
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20))

            for (ex, ey, ew, eh) in eyes:
                eye_roi = face_roi[ey:ey+eh, ex:ex+ew]  
                eye_roi = cv2.resize(eye_roi, (32, 32))  
                eye_roi = eye_roi / 255.0  
                eye_roi = eye_roi.reshape(1, 32, 32, 1)  
                
                prediction = model.predict(eye_roi)
                label = "Closed" if np.argmax(prediction) == 1 else "Open"
                print(f"Eye Status: {label}")
                
                color = (0, 0, 255) if label == "Closed" else (0, 255, 0)
                cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), color, 2)
                cv2.putText(frame, label, (x+ex, y+ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                if label == "Closed":
                    closed_eye_counter += 1
                else:
                    closed_eye_counter = 0

                if closed_eye_counter >= threshold:
                    cv2.putText(frame, "DROWSINESS ALERT!", (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                    print("DROWSINESS ALERT!")  
                    time.sleep(2)  
        
        cv2.imshow("Drowsiness Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Eye Status: Closed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Eye Status: Closed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Eye Status: Closed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Eye Status: Open
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Eye Status: Open
1/1